In [1]:
import tensorflow as tf

/home/profnick/anaconda3/envs/ztdl/lib/python3.5/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
sess = tf.Session()

In [4]:
import numpy as np

x_vals = np.array([1.,3.,5.,7.,9.])
x_data = tf.placeholder(tf.float32)
m_const = tf.constant(3.)
my_product = tf.multiply(x_data, m_const)
for x_val in x_vals:
    print(sess.run(my_product, feed_dict={x_data: x_val}))
    

3.0
9.0
15.0
21.0
27.0


<h2> Section 2 </h2>
<p> Multiplying a placeholder by two matrices and then perform additon</p>
<p>It is also important to note how the data will change shape as it passes through. We will feed in two NumPy arrays of size 3 x 5. We will multiply each matrix by a constant of size 5 x 1, which will result in a matrix of size 3 x 1. We will then multiply this by a 1 x 1 matrix resulting in a 3 x 1 matrix again. Finally, we add a 3 x 1 matrix at the end, as follows:</p>

In [15]:
sess = tf.Session()

In [16]:
# Create the dataa and feed it to the corresponding placeholder
my_array = np.array([[1.,2.,5.,7.,9.],
                    [-2.,0.,2.,4.,6.],
                    [-6.,-3.,0.,3.,6.]])
x_vals = np.array([my_array, my_array + 1])
x_data = tf.placeholder(tf.float32, shape=(3,5))

In [17]:
# We create the constants we will use the matrix multiplication and Adition
m1 = tf.constant([[1.],[0.],[-1],[2.],[4.]])
m2 = tf.constant([[2.]])
a1 = tf.constant([[10.]])

In [18]:
prod1 = tf.matmul(x_data, m1)
prod2 = tf.matmul(prod1, m2)
add1 = tf.add(prod2, a1)

In [19]:
for x_val in x_vals:
    print(sess.run(add1, feed_dict={x_data: x_val}))

[[102.]
 [ 66.]
 [ 58.]]
[[114.]
 [ 78.]
 [ 70.]]


<p><strong>NB: </strong>We have to declare the data shape and know the outcome shape of the operations before we run data through the graph. This is not always the case. There may be a dimension or two that we do not know beforehand, or some that can vary. To accomplish this, we designate the dimension or dimensions that can vary (or is unknown) as value None. For example, to make the prior data placeholder have an unknown amount of columns, we would write the following line:</p> 

In [13]:
x_data = tf.placeholder(tf.float32, shape=(3,None))

<h2> Section 3 </h2>

In [21]:
sess = tf.Session()

In [22]:
x_shape = [1,4,4,1]
x_val = np.random.uniform(size=x_shape)
x_data = tf.placeholder(tf.float32, shape=x_shape)

In [23]:
my_filter = tf.constant(0.25, shape = [2,2,1,1])
my_strides = [1,2,2,1]
mov_avg_layer = tf.nn.conv2d(x_data,my_filter,my_strides, 
                             padding='SAME', name='Moving_Avg_Window')

In [24]:
def custom_layer(input_matrix):
    input_matrix_squeezed = tf.squeeze(input_matrix)
    A = tf.constant([[1.,2.], [-1.,3.]])
    b = tf.constant(1.,shape=[2,2])
    temp1 = tf.matmul(A, input_matrix_squeezed)
    temp = tf.add(temp1,b) # Ax + b
    return tf.sigmoid(temp)

In [25]:
with tf.name_scope('Custom_Layer') as scope:
    custom_layer1 = custom_layer(mov_avg_layer)

print(sess.run(custom_layer1, feed_dict={x_data: x_val}))

[[0.94116414 0.8599201 ]
 [0.9520988  0.8517347 ]]


In [26]:
import matplotlib.pyplot as plt
%matplotlib inline

In [38]:
x_vals = tf.linspace(-1.,1.,500)
target = tf.constant(0.)

<p>L2 norm loss or the Euclidean loss </p>

In [29]:
l2_y_vals = tf.square(target - x_vals)
l2_y_out = sess.run(l2_y_vals)

<p>L1 norm loss or Absolute Loss</p>

In [31]:
l1_y_vals = tf.abs(target - x_vals)
l1_y_out = sess.run(l1_y_vals)

<p> Huber loss function</p>

In [34]:
delta1 = tf.constant(0.25) 
phuber1_y_vals = tf.multiply(tf.square(delta1), tf.sqrt(1. +  
                        tf.square((target - x_vals)/delta1)) - 1.) 
phuber1_y_out = sess.run(phuber1_y_vals) 
delta2 = tf.constant(5.) 
phuber2_y_vals = tf.multiply(tf.square(delta2), tf.sqrt(1. +  
                        tf.square((target - x_vals)/delta2)) - 1.) 
phuber2_y_out = sess.run(phuber2_y_vals)

In [43]:
x_vals = tf.linspace(-3., 5., 500) 
target = tf.constant(1.) 
targets = tf.fill([500,], 1.)

<p> Cross entropy loss General Case</p>

In [44]:
xentropy_y_vals = - tf.multiply(target, tf.log(x_vals)) - tf.multiply((1. - target), tf.log(1. - x_vals)) 
xentropy_y_out = sess.run(xentropy_y_vals)

<p> Sigmoid cross-entropy</p>

In [48]:
xentropy_sigmoid_y_vals = tf.nn.sigmoid_cross_entropy_with_logits(logits=x_vals, labels=targets) 
xentropy_sigmoid_y_out = sess.run(xentropy_sigmoid_y_vals)

<p>Softmax cross-entropy loss </p>

In [46]:
unscaled_logits = tf.constant([[1., -3., 10.]]) 
target_dist = tf.constant([[0.1, 0.02, 0.88]]) 
softmax_xentropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=unscaled_logits, labels=target_dist) 
print(sess.run(softmax_xentropy))

[1.1601256]


<h2> Section 3 Backpropagation </h2>

In [50]:
sess = tf.Session()

In [52]:
x_vals = np.random.normal(1,0.1,100)
y_vals = np.repeat(10.,100)
x_data = tf.placeholder(shape=[1], dtype=tf.float32)
y_target = tf.placeholder(shape=[1], dtype=tf.float32)
A = tf.Variable(tf.random_normal(shape=[1]))

In [54]:
my_output = tf.multiply(x_data, A)

In [55]:
loss = tf.square(my_output - y_target)

In [57]:
my_opt = tf.train.GradientDescentOptimizer(learning_rate=0.02)
train_step = my_opt.minimize(loss)

In [59]:
# Initialize your model variables
init = tf.initializers.global_variables()
sess.run(init)

In [61]:
for i in range(100):
    rand_index = np.random.choice(100)
    rand_x = [x_vals[rand_index]]
    rand_y = [y_vals[rand_index]]
    sess.run(train_step, feed_dict={x_data: rand_x, y_target: rand_y})
    if(i + 1) % 25 == 0:
        print('Step #' + str(i + 1 ) + ' A = ' + str(sess.run(A)))
        print('Loss= ' + str(sess.run(loss, feed_dict={x_data: rand_x, y_target: rand_y})))
        

Step #25 A = [9.747798]
Loss= [0.03228965]
Step #50 A = [9.805452]
Loss= [0.06202265]
Step #75 A = [10.084323]
Loss= [0.01429488]
Step #100 A = [9.810892]
Loss= [0.0095662]


In [62]:
from tensorflow.python.framework import ops 
ops.reset_default_graph() 
sess = tf.Session()

In [66]:
%config IPCompleter.greedy=True

In [63]:
x_vals = np.concatenate((np.random.normal(-1,1,50), np.random.normal(3,1,50)))
y_vals = np.concatenate((np.repeat(0.,50), np.repeat(1.,50)))
x_data = tf.placeholder(shape=[1], dtype=tf.float32)
y_data = tf.placeholder(shape=[1], dtype=tf.float32)
A = tf.Variable(tf.random_normal(mean=10, shape=[1]))

In [64]:
my_output = tf.add(x_data, A)

In [67]:
my_output_expanded = tf.expand_dims(my_output,0)
y_target_expanded = tf.expand_dims(y_target, 0)

In [69]:
init = tf.global_variables_initializer()
sess.run(init)

In [73]:
# Loss function
xentropy = tf.nn.sigmoid_cross_entropy_with_logits(labels=my_output_expanded,logits=y_target_expanded)

ValueError: Tensor("ExpandDims:0", shape=(1, 1), dtype=float32) must be from the same graph as Tensor("ExpandDims:0", shape=(1, 1), dtype=float32).

In [ ]:
my_opt = tf.train.GradientDescentOptimizer(0.05) 
train_step = my_opt.minimize(xentropy)

In [ ]:
for i in range(1400): 
    rand_index = np.random.choice(100) 
    rand_x = [x_vals[rand_index]] 
    rand_y = [y_vals[rand_index]] 
    sess.run(train_step, feed_dict={x_data: rand_x, y_target: rand_y}) 
    if (i + 1) % 200 == 0: 
        print('Step #' + str(i+1) + ' A = ' + str(sess.run(A))) 
        print('Loss = ' + str(sess.run(xentropy, feed_dict={x_data: rand_x, y_target: rand_y})))